In [320]:
import numpy as np
import pandas as pd
import re
import torch

from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

In [232]:
data = pd.read_csv('train_dataset_train.csv', sep=';')

In [247]:
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
import string
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer 
from nltk.tokenize import RegexpTokenizer

stopwords = stopwords.words('russian')
from tqdm import tqdm
from transformers import BertModel, BertTokenizer
from sklearn.model_selection import train_test_split

stop = ['привет','здравствуйте','добрый']
for word in stop:
    stopwords.append(word)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kirill/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2023-11-23 20:20:08.966311: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [236]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)


def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_punct(text):
    table= str.maketrans('','',string.punctuation)
    return text.translate(table)



def create_corpus(df):
    corpus=[]
    for tweet in tqdm(data['Текст инцидента']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stopwords))]
        corpus.append(words)
    return corpus

    

In [237]:
dirty_text = dirty_text.apply(lambda x : remove_html(x))
dirty_text = dirty_text.apply(lambda x : remove_punct(x))
dirty_text = dirty_text.apply(lambda x : remove_emoji(x))
dirty_text = dirty_text.apply(lambda x :remove_URL(x))
corpus=create_corpus(dirty_text)


for i in tqdm(range(0, len(corpus))):
    
    corpus[i] = " ".join(corpus[i])

data["clean_text"] = corpus

100%|██████████| 23128/23128 [00:00<00:00, 499538.41it/s]


In [251]:
X = data['clean_text']
y = data['Исполнитель']

df = data[['Исполнитель', 'clean_text']]

In [295]:
def prepare_input(text, tokenizer):
    """
    This function tokenizes the input text with the configured padding and truncation. Then,
    returns the input dictionary, which contains the following keys: "input_ids",
    "token_type_ids" and "attention_mask". Each value is a torch.tensor.
    :param cfg: configuration class with a TOKENIZER attribute.
    :param text: a numpy array where each value is a text as string.
    :return inputs: python dictionary where values are torch tensors.
    """
    inputs = tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=128,
        padding='max_length', # TODO: check padding to max sequence in batch
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long) # TODO: check dtypes
    return inputs


def collate(inputs):
    """
    It truncates the inputs to the maximum sequence length in the batch. 
    """
    mask_len = int(inputs["attention_mask"].sum(axis=1).max()) # Get batch's max sequence length
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs


class CustomDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.texts = df['clean_text'].values
        self.labels = df['Исполнитель'].values
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        output = {}
        output["inputs"] = prepare_input(self.texts[item], self.tokenizer)
        output["labels"] = torch.tensor(self.labels[item], dtype=torch.float) # TODO: check dtypes
        return output

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
X = df.loc[:, df.columns != "Исполнитель"]
y = df.loc[:, df.columns == "Исполнитель"]

for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
    df.loc[valid_index, "fold"] = i
    
df.head()

In [297]:
fold = 0
train_folds = df[df['fold'] != fold].reset_index(drop=True)
valid_folds = df[df['fold'] == fold].reset_index(drop=True)
valid_labels = valid_folds['Исполнитель'].values


In [299]:
import multiprocessing

In [304]:
# ======== DATASETS ==========
train_dataset = CustomDataset(train_folds, tokenizer)
valid_dataset = CustomDataset(valid_folds, tokenizer)

# ======== DATALOADERS ==========
train_loader = DataLoader(train_dataset,
                            batch_size= 32, # TODO: split into train and valid
                            shuffle=True,
                            num_workers= multiprocessing.cpu_count(), pin_memory=True, drop_last=True)
valid_loader = DataLoader(valid_dataset,
                            batch_size=16,
                            shuffle=False,
                            num_workers=multiprocessing.cpu_count(), pin_memory=True, drop_last=False)


